<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/scripts/elinor_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose

This file shows the steps we took to sample and create the annotation dataset.

## Connect with Google drive to access data 

In order to access the data, you first need to create a shortcut of the data folder to your own Gdrive. If you've been granted editing rights, you should be able to edit the content of the folder, i.e. add, move and delete data, create and rename folders, etc.

In [1]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# redirect the working directory of this script to the data folder
%cd /content/drive/MyDrive/data/

/content/drive/MyDrive/data


## Load data

In [9]:
import tqdm as tqdm
import os
import pandas as pd

folder_path = "filtered2"

In [10]:
dfs = []

# loop through files 
for filename in os.listdir(folder_path):
    # if csv file, load and add to dfs  
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

# combine files in df
df = pd.concat(dfs, ignore_index=True)

## Inspect data


In [20]:
print(df.shape)
df.info()

(1247840, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247840 entries, 0 to 1247839
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1247840 non-null  object 
 1   artikel_id  1247840 non-null  object 
 2   name        1247840 non-null  object 
 3   jahrgang    1246781 non-null  object 
 4   datum       1247825 non-null  float64
 5   ressort     1121457 non-null  object 
 6   titel       1166481 non-null  object 
 7   untertitel  571720 non-null   object 
 8   text        1247840 non-null  object 
dtypes: float64(1), object(8)
memory usage: 85.7+ MB


In [41]:
# remove first column 
df1 = df.drop("Unnamed: 0", axis=1)

## Create a random subset of the data 

In [31]:
# size of subset we want 
number = 100

In [100]:
perc = number/df1.shape[0]
df_subset = df1.sample(frac=perc, random_state=42)

In [97]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 141433 to 1157830
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   artikel_id  100 non-null    object 
 1   name        100 non-null    object 
 2   jahrgang    100 non-null    object 
 3   datum       100 non-null    float64
 4   ressort     89 non-null     object 
 5   titel       92 non-null     object 
 6   untertitel  37 non-null     object 
 7   text        100 non-null    object 
dtypes: float64(1), object(7)
memory usage: 7.0+ KB


## Adjust format for export

In [101]:
# remove square brackets 
df_subset["text"] = df_subset["text"].apply(lambda x: x[1:-1] if (isinstance(x, str) and x.startswith("[") and x.endswith("]")) else x)
# remove backward slashes
df_subset['text'] = df_subset['text'].str.replace('\\', '', regex = False)


In [102]:
# function to split strings to list
def split_to_list(s):
    return s.split("', '")

In [103]:
# apply function to whole column
df_subset['text'] = df_subset['text'].apply(split_to_list)

In [109]:
# add new line to separate paragraphs 
df_subset['text'] = df_subset['text'].apply(lambda x: [char+'\n' for char in x])

## Export as csv

In [111]:
output_path = "elinor"

In [113]:
df_subset.to_csv(output_path+"/annotation_test.csv", index=False, sep="\t", header=True)